In [ ]:
import sys
sys.path.append('../..')
#sys.path.append( '/home/cactuskid13/miniconda3/pkgs/')
print(sys.path)

In [ ]:
from pyprofiler.utils import hashutils
import ete3
import random
from pyprofiler.utils import config_utils
import pyprofiler.utils.goatools_utils as goa
import pyprofiler.utils.hashutils as hashutils
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import pyprofiler.profiler as profiler
import pandas as pd
import itertools
import redis
##to get the mapping of oma hogs to cogs to interactions in specific species I used dask distributed and a redis server
#you may need to get these up and running for you own cluster configuration before this notebook will work for you
import dask
import scipy
from dask import dataframe as dd
import pickle
from bloom_filter2 import BloomFilter
from sklearn.model_selection import train_test_split


In [ ]:
#lets load a compiled db containing the OMA root HOGs into a profiler oject 
p = profiler.Profiler(lshforestpath = '/scratch/dmoi/datasets/all/newlshforest.pkl' , hashes_h5='/scratch/dmoi/datasets/birds/all/hashes.h5' , mat_path= None, oma = '/scratch/dmoi/datasets/OMA/apr2021/OmaServer.h5', tar= None , nsamples = 256 , mastertree = '/scratch/dmoi/datasets/birds/all_test_master_tree.pkl')

In [ ]:
def grabHog(ID, verbose = True):
    try:
        entry = p.db_obj.entry_by_entry_nr(p.db_obj.id_resolver.resolve(ID))
        return entry[4].decode() , entry
    except:
        return np.nan,np.nan
#map to OMA HOGs

In [ ]:
humap = '/scratch/dmoi/datasets/humap_PPI/humap2_ppis_ACC_20200821.pairsWprob'
calc_humap = False
if calc_humap == True:
    #load humap data
    humap_df = pd.read_table(humap, header = None)
    print(humap_df)
    humap_df = humap_df[humap_df[2] > .75 ]
    mapper = set( list(humap_df[1]) + list(humap_df[0]) )
    mapper = { protid: grabHog(protid) for protid in mapper }
    humap_df['hog1'] = humap_df[1].map(mapper)
    humap_df['hog2'] = humap_df[0].map(mapper)
    humap_df['hogid_1'] = humap_df['hog1'].map(lambda x:x[0])
    humap_df['hogid_2'] = humap_df['hog2'].map(lambda x:x[0])
    humap_df = humap_df.dropna()
    humap_df['fam1'] = humap_df['hog1'].map( lambda x :   p.hogid2fam(x[1]) )
    humap_df['fam2'] = humap_df['hog2'].map( lambda x :   p.hogid2fam(x[1]) ) 
    humap_df = humap_df.dropna()
    humap_df.fam1 = humap_df.fam1.map(int)
    humap_df.fam2 = humap_df.fam2.map(int)
    print(len(humap_df))
    humap_df.to_csv(humap+'hogmapped.csv')
else:
    humap_df = pd.read_csv(humap+'hogmapped.csv')

In [ ]:
humap_pairs = humap_df

In [ ]:
profiles = {}
calc_hogs_humap = False
if calc_hogs_humap == True:
    allhogs = set([])
    allhogs = allhogs.union( set(humap_df.fam1.unique() ) )
    allhogs = allhogs.union( set(humap_df.fam2.unique() ) )
    print(len(allhogs))
    for fam in allhogs:
        print(fam)
        try:
            prof = p.return_profile_OTF(fam)
        except:
            print('err', fam)
        print(prof)
        profiles.update(prof)

In [ ]:
save_Hogs_humap = False
if save_Hogs_humap == True:
    with open(humap + 'gold_standard_profiles.pkl' , 'wb') as profiles_out:
        profiles_out.write(pickle.dumps(profiles))

In [ ]:
with open(humap + 'gold_standard_profiles.pkl' , 'rb') as profiles_out:
    humap_profiles = pickle.loads(profiles_out.read())

In [ ]:
humap_df = pd.DataFrame.from_dict(humap_profiles , orient = 'index')
print(humap_df.head())

In [ ]:
#################begin building the string dataset ###################################

In [ ]:
filter_coglinks = False
if filter_coglinks == True:
    coglink_df = dd.read_csv('/scratch/dmoi/datasets/STRING/COG.links.detailed.v11.5.txt', blocksize=25e6 , header = 0, sep = ' ')
    print(coglink_df)
    print(coglink_df.columns)
    dropcols = ['neighborhood', 'fusion', 'cooccurence', 'combined_score' ]
    coglink_df = coglink_df.drop(columns = dropcols)
    coglink_df['score'] = coglink_df.coexpression + coglink_df.experimental +coglink_df.database+ coglink_df.textmining
    coglink_df= coglink_df[coglink_df.score>1000]
    coglink_df = coglink_df.compute()
    print(coglink_df.head())

In [ ]:
#map the interacting cogs to the proteins
compute_grabcogs = False
if compute_grabcogs == True:
    grabcogs = set( list(coglink_df.group1.unique()) + list(coglink_df.group2.unique()) )
    grabcogs= list(grabcogs)
    COGmapings_df = dd.read_csv('/scratch/dmoi/datasets/STRING/COG.mappings.v11.5.txt', blocksize=25e6 , header = 0, sep = '\t')
    COGmapings_df = COGmapings_df.set_index('orthologous_group')
    COGmapings_df.astype(str)
    COGmapings_df['##protein'].map( lambda x : x.strip() )
    COGmapings_df['species'] = COGmapings_df['##protein'].map( lambda x : x.split('.')[0] )
    COGmapings_df['COG'] = COGmapings_df.index
    COGmapings_df = COGmapings_df.loc[grabcogs]
    COGmapings_df = COGmapings_df.compute()
    print(COGmapings_df.head())

In [ ]:
#only take the proteins in our cogs of interest
if compute_grabcogs == True:
    grabprots =list(COGmapings_df['##protein'].unique())
    print(len(grabprots))
    with open('/scratch/dmoi/datasets/STRING/COG.links.detailed.v11.5.txt' + '.grabcogs.txt', 'w') as protsout:
        protsout.write(''.join([ p + '\n' for p in grabcogs ]) )
    with open('/scratch/dmoi/datasets/STRING/COG.mappings.v11.5.txt' + '.grabprots.txt' , 'w') as protsout:
        protsout.write(''.join([ p + '\n' for p in grabprots ]) )
else:
    with open('/scratch/dmoi/datasets/STRING/COG.links.detailed.v11.5.txt' + '.grabcogs.txt', 'r') as protsout:
        grabcogs = [ cog for cog in protsout.readlines()]
    with open('/scratch/dmoi/datasets/STRING/COG.mappings.v11.5.txt' + '.grabprots.txt' , 'r') as protsout:
        grabprots = [ prot for prot in protsout.readlines()]

In [ ]:
calc_mappers = False
rdb = redis.Redis(host='10.202.12.174', port=6379, db=0)
if calc_mappers == True:
    count = 0
    for i,r in COGmapings_df.iterrows():
        rdb.set(r['##protein'], i)
        count+=1
        if count < 10:
            print(i+'\n',r)
        if count%1000000==0:
            print(count/len(COGmapings_df))

In [ ]:
maphogs = False
if maphogs == True:
    #mapping each string cog to an oma hog by selecting a member of the cog
    rdb = redis.Redis(host='10.202.12.174', port=6379, db=0)
    hogmap = {}
    for i,prot in enumerate(grabprots):
        if i % 100000 == 0 :
            print(i/len(grabprots))
        cog = rdb.get(prot)
        if cog not in hogmap:
            mapped =  grabHog(prot)
            #retry until something maps
            if mapped[0] != np.nan and type(mapped[0]) == str :
                if len(mapped[0])>1 :
                    hogmap[cog] = mapped
    with open('stringhogmap.pkl' , 'wb')as hogmapout:
        hogmapout.write(pickle.dumps(hogmap))
else:
    with open('stringhogmap.pkl' , 'rb')as hogmapout:
        hogmap = pickle.loads(hogmapout.read())


In [ ]:
print(len(hogmap))
for i, key in enumerate(hogmap):
    if i < 10:
        print(key, hogmap[key])

In [ ]:
#add the HOGs to the COGdf
#grab the corresponding profiles
compile_final_cogdf = False
if compile_final_cogdf == True:
    print(len(coglink_df))
    try:
        coglink_df.group1  = coglink_df.group1.map( lambda x : x.encode())
        coglink_df.group2  = coglink_df.group2.map( lambda x : x.encode())
    except:
        pass
    coglink_df['hog1'] = coglink_df.group1.map(hogmap)
    coglink_df['hog2'] = coglink_df.group2.map(hogmap)
    coglink_df=coglink_df.dropna()
    print(len(coglink_df))
    print(coglink_df.head())
    coglink_df['hogid_1'] = coglink_df['hog1'].map(lambda x:x[0])
    coglink_df['hogid_2'] = coglink_df['hog2'].map(lambda x:x[0])
    coglink_df['fam1'] = coglink_df['hog1'].map( lambda x :   p.hogid2fam(x[1]) )
    coglink_df['fam2'] = coglink_df['hog2'].map( lambda x :   p.hogid2fam(x[1]) ) 
    coglink_df.fam1 = coglink_df.fam1.map(int)
    coglink_df.fam2 = coglink_df.fam2.map(int)
    stringHOGs = set(coglink_df.fam1.unique()).union(set(coglink_df.fam2.unique()))
    print(len(stringHOGs))
    print(coglink_df)
    coglink_df.to_csv('STRINGCOGS2OMAHOGS.csv')
else:
    coglink_df = pd.read_csv('STRINGCOGS2OMAHOGS.csv')

In [ ]:
stringPairs = coglink_df

In [ ]:
#derive explicit profiles for our hogs of interest in string
calc_hogs_string = False
stringprofiles = {}
if calc_hogs_string == True:
    print('profiles to calclulate',len(stringHOGs))
    for i,fam in enumerate(stringHOGs):
        if i % 100 ==0:
            print(i)
        try:
            prof = p.return_profile_OTF(fam)
            stringprofiles.update(prof)
        except:
            print('err',fam)

In [ ]:
if calc_hogs_string == True:
    with open('/scratch/dmoi/datasets/STRING/' + 'gold_standard_profiles.pkl' , 'wb') as profiles_out:
        profiles_out.write(pickle.dumps(stringprofiles))

In [ ]:
with open('/scratch/dmoi/datasets/STRING/' + 'gold_standard_profiles.pkl' , 'rb' )as profiles_out:
    stringprofiles = pickle.loads(profiles_out.read())

In [ ]:
string_df = pd.DataFrame.from_dict(stringprofiles , orient='index')

In [ ]:
#make the profiles for this small set of HOGs
for i, key in enumerate(stringprofiles):
    if i < 10:
        print(key,stringprofiles[key])

In [1]:
#now we have profiles for all HUMAP and COG interactions
#String has interactions from each COG in different species.
#We need a way to check for the presence of interaction within a species for a COG
#for this we will create a bloom filter with all the interactions between our cogs

calc_filter = True

In [2]:
if calc_filter == True:
    from dask.distributed import fire_and_forget
    from dask.distributed import Client, Variable , Queue , Lock ,LocalCluster
    from dask_jobqueue import SLURMCluster
    from dask.distributed import  utils_perf
    from dask.distributed import Client, LocalCluster
    import dask
    import redis
    from bloom_filter2 import BloomFilter
    import lzma
    from dask import dataframe as dd
    distributed = True

/scratch/dmoi/condaenvs/ML2/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [3]:
if calc_filter == True:
    if distributed == True:
        NCORE = 4
        print('deploying cluster')
        cluster = SLURMCluster(
            walltime='4:00:00',
            n_workers = NCORE,
            cores=NCORE,
            processes = NCORE,
            interface='ib0',
            memory="120GB",
            env_extra=[
            'source /scratch/dmoi/miniconda/etc/profile.d/conda.sh',
            'conda activate ML2'
            ],
            scheduler_options={'interface': 'ens2f0' },
            #extra=["--lifetime", "3h55m", "--lifetime-stagger", "4m"]
        )
        print(cluster.job_script())

    else:
        cluster = LocalCluster()
        client = Client(cluster)

deploying cluster
#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -n 1
#SBATCH --cpus-per-task=4
#SBATCH --mem=112G
#SBATCH -t 4:00:00
source /scratch/dmoi/miniconda/etc/profile.d/conda.sh
conda activate ML2
/scratch/dmoi/condaenvs/ML2/bin/python -m distributed.cli.dask_worker tcp://10.203.100.107:38831 --nthreads 1 --nprocs 4 --memory-limit 27.94GiB --name dummy-name --nanny --death-timeout 60 --interface ib0 --protocol tcp://



In [4]:
if calc_filter == True:
    if distributed == True:
        print(cluster)
        cluster.scale(jobs = 100)
        print(cluster.dashboard_link)
        client = Client(cluster , timeout='450s' , set_as_default=True )

SLURMCluster(13da5ef2, 'tcp://10.203.100.107:38831', workers=0, threads=0, memory=0 B)
http://10.203.100.107:8787/status


In [5]:
#find which species each of the cogs has an interaction in
if calc_filter == True:
    
    #link_df = dd.read_csv('/scratch/dmoi/datasets/STRING/protein.physical.links.detailed.v11.5.txt', blocksize=100e6 , header = 0, sep = ' ')
    link_df = dd.read_csv('/scratch/dmoi/datasets/STRING/protein.links.detailed.v11.5.txt',  blocksize=100e6 , header = 0, sep = ' ')
    print(link_df)
    

Dask DataFrame Structure:
                  protein1 protein2 neighborhood fusion cooccurence coexpression experimental database textmining combined_score
npartitions=12990                                                                                                               
                    object   object        int64  int64       int64        int64        int64    int64      int64          int64
                       ...      ...          ...    ...         ...          ...          ...      ...        ...            ...
...                    ...      ...          ...    ...         ...          ...          ...      ...        ...            ...
                       ...      ...          ...    ...         ...          ...          ...      ...        ...            ...
                       ...      ...          ...    ...         ...          ...          ...      ...        ...            ...
Dask Name: read-csv, 12990 tasks


In [14]:
#compute bloom filters for protein pairs
@dask.delayed
def mapcogs(df ):
    rdb = redis.Redis(host='10.202.12.174', port=6379, db=0)
    if type( df ) == tuple:
        df = df[0]
    protlist1 = list(df.protein1.map(lambda x:str(x).strip()))
    protlist2 = list(df.protein2.map(lambda x:str(x).strip()))
    protlist = list(set(protlist1+protlist2))
    data = rdb.mget(protlist)
    mapper = dict(zip(protlist, data) )
    df['COG1'] = df.protein1.map(mapper)
    df['COG2'] = df.protein2.map(mapper)
    df = df.dropna()
    df['COG1'] = df.COG1.map(lambda x:str(x).replace("b",'').replace("'",'').strip() )
    df['COG2'] = df.COG2.map(lambda x:str(x).replace("b",'').replace("'",'').strip() )
    df['species'] = df.protein1.map(lambda x:x.split('.')[0])
    df['coglinks'] = df.COG1 + '_' + df.COG2 + '_' + df.species
    ret = set(df.coglinks.unique())
    return ret
@dask.delayed
def return_filter(coglinks, verbose = True):
    if type( coglinks ) == tuple:
        coglinks = coglinks[0]
    b=BloomFilter(max_elements=10**8, error_rate=0.001 ,start_fresh = True)
    for p in coglinks:
        b.add( p )
    retlen = len(coglinks)
    return   b , retlen

@dask.delayed
def sumfilter(f1,f2, total ):
    if type( f1 ) == tuple:
        f1 = f1[0]
    if type( f2 ) == tuple:
        f2 = f2[0]
    f3 = f1.__ior__(f2)
    return f3 , total

In [16]:
def treesum(totalfilter):
    print(len(totalfilter))
    while len(totalfilter)>1:
        next_round= []
        for i in range(0,len(totalfilter),2):
            if i+1 < len(totalfilter):
                next_round.append( sumfilter( totalfilter[i][0] , totalfilter[i+1][0] , totalfilter[i][1]+totalfilter[i+1][1]  ) )
        if len(totalfilter) % 2 !=0:
            next_round.append(totalfilter[-1])
        totalfilter = next_round
        print(len(totalfilter))
    return totalfilter

if calc_filter == True:
    b=BloomFilter(max_elements=10**8, error_rate=0.001 ,start_fresh = True)
    partitions  = link_df.to_delayed()
    print('map cogs')
    res1 = [ mapcogs(p) for p in partitions ]
    print('done')
    print('make filters')
    res2 = [ return_filter(p) for p in res1 ]
    finals =[]
    for chunk in range(int(len(res2)/1024)+1):
        print(chunk*1024)
        res3 = res2[chunk*1024:(chunk+1)*1024]
        res4 = treesum(res3)
        res4 = dask.compute(res4)
        print(res4)
        finals.append(res4[0])
    

In [19]:
print(finals)
peel_finals = [r[0] for r in finals]
resfinal = treesum(peel_finals)
resfinal = dask.compute(peel_finals)

[[(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 122010548)], [(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 220957577)], [(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 169856375)], [(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 119428878)], [(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 126517031)], [(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 185650531)], [(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 199204385)], [(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 241435144)], [(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 228547216)], [(BloomFilter(ideal_num_elements_n=100000000,

In [ ]:
if calc_filter == True:
    resfinal = dask.compute(totalfilter)

In [ ]:
#client.cancel(resfinal)

In [24]:
if calc_filter == True:
    import pickle
    with open('bloomfinal_big.pkl' , 'wb' ) as finalout:
        finalout.write(pickle.dumps(resfinal))

In [25]:
with open('bloomfinal_big.pkl' , 'rb' ) as finalout:
    resfinal = pickle.loads(finalout.read()) 
cogfilter =  resfinal[0][0][0]

In [26]:
print('COG1756_COG0088_4113' in cogfilter )

True


In [ ]:
#lets try it out. we should be able to find in which species two cogs interact with the bloom filter
cog1='COG0088'
cog2 ='COG1756'
coglink = cog1 + '_' + cog2 + '_'

In [ ]:
#test out to find the species for a cog pair
coglinks_species = [ coglink+spec.name  for spec in p.tree.get_leaves()]
species = [ spec.name  for spec in p.tree.get_leaves()]
checklinks = [ coglink+spec in cogfilter for spec in coglinks_species ]
species_set = [s for s in species]
links = [ l for l,c in list(zip(coglinks_species,checklinks))  if c== True  ]
species_set = [ s for s,c in list(zip(species_set,checklinks))  if c== True  ]
print(links)
print(species_set)

In [ ]:
#mapfam to matrow
humap_fam_map= { f:i for i,f in enumerate(humap_df.index)}
humap_profilemat = np.vstack(humap_df.mat)
print(humap_profilemat.shape)
string_fam_map = { f:i for i,f in enumerate(string_df.index)}
string_mat = np.vstack(string_df.mat)
print(string_mat.shape)
#train test split
Datasets = {}
for label,df,mapping,profilemat in [  ('string', stringPairs, string_fam_map ,string_mat ) , ('humap',humap_pairs, humap_fam_map , humap_profilemat) ]: 
    keys = set(mapping.keys())
    entry1 = [ f in keys for f in df.fam1]
    df = df.iloc[entry1]
    entry2 = [ f in keys for f in df.fam2]
    df = df.iloc[entry2]
    msk = np.random.rand(len(df)) < 0.8
    df_train = df.iloc[msk]
    df_test = df.iloc[~msk]
    Datasets[label]={'Train':df_train,'Test':df_test , 'mapping': mapping , 'mat':profilemat }
    print(label)
    print('train',len(df_train))
    print('test',len(df_test))

In [ ]:

def chunks(df, n):
    for i in range(0, len(df), n):
        yield df.iloc[i:i + n]
def generateXYchunk(explicit_profiles, goldstandardDF,fam_map,  nsamples=100, posi_percent = .5):
    #shuffle
    goldstandardDF = goldstandardDF.sample(frac=1)
    for chunkdf in chunks(goldstandardDF , int( nsamples*posi_percent)):
        #negatives drawn from the overall dataset
        X = np.hstack([ np.vstack([profilemat[fam_map[f]] for f in chunkdf.fam1]) , np.vstack([profilemat[fam_map[f]] for f in chunkdf.fam2]) ] )
        Y = [1]* X.shape[0]
        neg1 = list(goldstandardDF.sample(n = int(nsamples*(1-posi_percent))).fam1)
        neg2 = list(goldstandardDF.sample(n = int(nsamples*(1-posi_percent))).fam2)
        
        if len(neg1)>0:
            mixchunk = np.hstack([np.vstack([profilemat[fam_map[f]] for f in neg1]),np.vstack([profilemat[fam_map[f]] for f in neg2])])
            Y =np.hstack([[0]* mixchunk.shape[0] , Y])
            X= np.vstack([mixchunk,X])    
        #positive samples
        yield X, Y

In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

def ROC_curve(y_data, label = None):
    plt.figure(figsize=(10,10))
    plt.plot([0, 1], [0, 1], 'k--')
    for l in y_data:
        print(l)
        y_pred_grd = y_data[l]['Ytrue']
        y_test = y_data[l]['Ypred']
        fpr, tpr, _ = roc_curve(   y_pred_grd ,y_test)
        plt.plot(fpr, tpr, label=l + 'auc'+ str(auc(fpr, tpr) ))
        plt.xlabel('False positive rate')
        plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(loc='best')
    if label:
        plt.savefig( label +'_ROC.svg' )
    plt.show()
    
    
    plt.figure(figsize=(10,10))
    for l in y_data:
        y_pred_grd = y_data[l]['Ytrue']
        y_test = y_data[l]['Ypred']
        precision, recall, thresholds = precision_recall_curve( y_pred_grd, y_test)
        plt.plot( recall, precision , label= l )
        plt.ylabel('Precision')
        plt.xlabel('Recall')
    
    plt.title('PR curve')
    plt.legend(loc='best')
    if label:
        plt.savefig( label +'_PR.svg' )
    plt.show()
    
def ROC_curve_single(y_test, y_pred_grd):
    fpr_grd, tpr_grd, _ = roc_curve(y_test, y_pred_grd)

    plt.plot([0, 1], [0, 1], 'k--')

    fpr, tpr, _ = roc_curve(y_test, y_pred_grd)
    plt.plot(fpr, tpr, label='single')
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')

    plt.title('ROC curve')
    plt.legend(loc='best')
    plt.show()

    precision, recall, thresholds = precision_recall_curve(  y_test , y_pred_grd)
    plt.plot(precision, recall , label='single')
    plt.xlabel('Precision')
    plt.ylabel('Recall')

    plt.title('PR curve')
    plt.legend(loc='best')
    plt.show()


In [ ]:
#try a vanilla deep NN
from keras.layers import Dense, Input
from keras.models import Sequential
import os
import tensorflow as tf
from keras.models import load_model

overwrite = True
train_dnn = False

if train_dnn == True:
    for dataset in Datasets:
        modelpath = './'+dataset+'_dropout_DNN.h5'
        callbacks = [
        tf.keras.callbacks.TensorBoard(log_dir= modelpath+'.logs'),
        tf.keras.callbacks.ModelCheckpoint(filepath=modelpath)
        ]
        print(dataset)
        df_train = Datasets[label]['Train']
        fam_map = Datasets[label]['mapping']
        profilemat = Datasets[label]['mat']
        if os.path.exists(modelpath) and overwrite == False:
            model = load_model(modelpath)
        else:
            model = Sequential()
            model.add(Dense(units=100, activation='sigmoid', input_dim=profilemat.shape[1]*2))
            model.add(tf.keras.layers.Dropout( .2 , seed=42 ))
            model.add(Dense(units=30, activation='sigmoid' ) )
            model.add(Dense(units=1, activation='sigmoid' ) )
        model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
        generator = generateXYchunk(profilemat, df_train, fam_map , posi_percent= .5 , nsamples = 50 )
        sample = next(generator)
        model.fit(itertools.cycle(generator) , steps_per_epoch = 300 , epochs = 50, callbacks=callbacks)
        # Save the model
        model.save(modelpath)



In [ ]:
#normal distance metrics to try

from scipy.spatial.distance import euclidean , hamming, jaccard
from sklearn.covariance import empirical_covariance
from scipy.stats import pearsonr
from keract import get_activations
def pearsonR(v1,v2):
        return -pearsonr(v1,v2)[0]

visualize = False    
if visualize == True:
    for label in Datasets:
        print(label)
        df_test = Datasets[label]['Test']
        fam_map = Datasets[label]['mapping']
        profilemat = Datasets[label]['mat']
        ydata =  {}

        for func, name in [ (euclidean, 'Euclidean' ), (hamming,'Hamming') , (jaccard,'Jaccard') , (pearsonR,'Pearson') ]:
            print(name)
            generator = generateXYchunk(profilemat, df_test, fam_map , posi_percent= .5 , nsamples = 50 )
            #test all the easy metrics
            ypreds = []
            ytruth = []
            for X,y in generator:
                #distances
                x1 = X[:,0:int(X.shape[1]/2)]
                x2 = X[:,int(X.shape[1]/2):]
                predictions = np.array([ -func(x1[r,:],x2[r,:] ) for r in range(x1.shape[0]) ])
                ypreds.append(predictions)
                ytruth.append(y)
            ytest= np.hstack(ytruth)
            ypred = np.hstack(ypreds)
            ydata[name] = { 'Ypred': ypred , 'Ytrue':ytest} 
        #get DNN values
        print('DNN')
        generator = generateXYchunk(profilemat, df_test, fam_map , posi_percent= .5 , nsamples = 50 )
        mats = [(x,y) for x,y in generator ]
        y_test = np.hstack([y for x,y in mats])
        modelpath = './'+label+'_dropout_DNN.h5'
        if os.path.exists(modelpath):
            model = load_model(modelpath)


        ypred = np.vstack([ model.predict(x) for x,y in mats])


        #activations = [ get_activations(model, x , auto_compile=True) for x,y in mats] 
        #activations = activations[list(activations.keys())[0]]
        #print( activations.shape)
        #n_samples = activations.shape[0]
        #activations = np.sum(activations,axis =0)/n_samples
        #representations = np.sum(X_test, axis = 0)

        #print(activations.shape)

        #np.save(modelpath + 'activation.np' , activations)
        #np.save(modelpath + 'representation.np', representations)

        ydata['DNN'] ={ 'Ypred': ypred , 'Ytrue':ytest}
        #plot ROC
        print(ydata)
        ROC_curve(ydata , label = label)
        plt.show()


In [ ]:
#######################being the graph NN part of the paper #####################################

In [ ]:

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.nn import ChebConv
from torch_geometric.nn import  to_hetero
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.data import HeteroData ,InMemoryDataset
#create graphs on the fly to represent pairs of profiles

In [ ]:
import dendropy
taxnwk = '/scratch/dmoi/datasets/birds/all_test_master_tree.nwk'
with open( 'taxtree.nwk' , 'w') as treeout:
    treeout.write(p.tree.write())
dendrotree = dendropy.Tree.get(
        data=p.tree.write(format=3),
        rooting="force-rooted",
        suppress_internal_node_taxa=False,
        schema='newick')

In [ ]:
nodes = set([ n.name for n in p.tree.traverse() ])
dendrotree_nodes = set([str(n.taxon.label) if n.taxon else '-1' for n in dendrotree.nodes()] )

In [ ]:
print(len(nodes))
print(len(dendrotree_nodes))
print(len(nodes.intersection(dendrotree_nodes)))

In [ ]:
profile_mapper = { n.name:i for i,n in enumerate(p.tree.traverse()) }

In [ ]:
#setup the internal nodes for the fitch algo
for i,l in enumerate(dendrotree.nodes()):
    l.event = {}
    l.scores = {}
    l.symbols = None
    l.char= None
    l.matrow = i

In [ ]:
profile_mapper = { (n.taxon.label if n.taxon else '-1'):n.matrow for n in  dendrotree.nodes() }

In [ ]:
import smallpars
#we're checking for interaction in a subset of species and propagating up
allowed_symbols =set([0,1])
transition_dict = { (c[0],c[1]):i for i,c in enumerate(itertools.permutations(allowed_symbols,2) ) }
def calc_interaction_on_taxonomy(cog1,cog2,treein, bloom , verbose = False):
    #set interaction states
    #look for interactions in bloom
    tree = copy.deepcopy(treein)
    for i,l in enumerate(tree.leaf_nodes()):
        l.event = {}
        l.scores = {}
        l.symbols = {}
        l.scores = { c:10**10 for c in allowed_symbols }
        if cog1+'_'+cog2+'_'+l.taxon.label in bloom:
            if verbose == True:
                print(l.taxon.label)
            l.char= 1
            l.symbols = {1}
        else:
            l.char= 0
            l.symbols = {0}
        l.scores[l.char] = 0
    t = smallpars.calculate_small_parsimony(tree ,allowed_symbols, transition_dict)
    labels = np.array( [ n.char for n in t.nodes() ] )
    return  labels

In [ ]:
def tree2Single_sparse_graph_updown(tree):
    N = len(tree.nodes())
    #mimic the fitch algo
    #propagate up and down in separate graphs
    index_up = np.vstack([ [n.matrow, c.matrow ] for n in tree.nodes() for c in n.child_nodes()])
    index_down = np.vstack([ [c.matrow, n.matrow ] for n in tree.nodes() for c in n.child_nodes()])
    
    connectmat_up = scipy.sparse.lil_matrix(( N ,  N ) )
    connectmat_down = scipy.sparse.lil_matrix(( N ,  N ) )
    
    connectmat_up[index_up[:,0],index_up[:,1]] = 1 
    connectmat_down[index_down[:,0],index_down[:,1]] = 1 

    diag = [[n,n] for n in range(N)]
    connectmat_diag=scipy.sparse.lil_matrix(( N ,  N ) )
    connectmat_diag[diag,diag] = 1 
    
    ntime = np.array([ n.distance_from_root() for n in tree.nodes()])
    mtime = np.amax(ntime)
    ntime/=mtime
    
    levels = np.array([ n.level() for n in tree.nodes() ] , dtype='double')
    levels /= np.amax(levels)
    
    Norm_nchild= np.array( [ len(n.child_nodes()) for n in tree.nodes() ] ,dtype='double' )
    mchild =np.amax(Norm_nchild)
    Norm_nchild/=mchild 
    
    Norm_nsister= np.array( [ len(n.sister_nodes()) for n in tree.nodes() ] ,dtype='double' )
    msis =np.amax(Norm_nsister)
    Norm_nsister/=msis    
    
    template_features = np.stack([ntime ,  Norm_nchild , Norm_nsister ]).T    
    return connectmat_up, connectmat_down, connectmat_diag, template_features



In [ ]:
import copy
def getmrca(treein,taxset):
    tree = copy.deepcopy(treein)
    n = tree.mrca(taxon_labels=taxset)
    subtree = dendropy.Tree(seed_node=n)
    taxset = set([ t.taxon.label if n.taxon else '-1'  for t in subtree.nodes()])
    matrows = [ t.matrow for t in subtree.nodes()]
    return taxset, matrows , n

In [ ]:
def sparse2pairs(sparsemat, matrows = None):
    if matrows :
        sparsemat = sparsemat[matrows,:]
        sparsemat = sparsemat[:,matrows]
    sparsemat = scipy.sparse.find(sparsemat)
    return np.vstack([sparsemat[0],sparsemat[1]])

In [ ]:
def create_data_updown( q, iolock, tree, coglinkdf, bloom, profiles , taxindex , posi_percent = .5 , verbose = False, loop= True):
        connectmat_up, connectmat_down, connectmat_diag, template_features = tree2Single_sparse_graph_updown(tree)
        
        num_total_nodes = template_features.shape[0]
        with iolock:
            print('dataset size' , len(coglinkdf))
        
        lineiterator = coglinkdf.iterrows()
        N = len(tree.nodes())
        
        #grab signal from the downward propagation
        leafnodes = np.array([n.matrow for n in tree.leaf_nodes()])
        overview_connect = scipy.sparse.lil_matrix(( N , 2 ) )
        overview_connect[leafnodes,0] = 1
        allfams = list(set(coglinkdf.fam1.unique()).union( set(coglinkdf.fam2.unique() ) ))
        while True:
            toss = scipy.stats.bernoulli.rvs(posi_percent, loc=0, size=1, random_state=None)
            if verbose == True:
                print('posi/nega',toss)
            if toss == 0:
                fam1 = random.choice(allfams)
                fam2 = fam1
                while fam1 == fam2:
                    fam2 = random.choice(allfams)
                labels = np.zeros((template_features.shape[0],))
            else:
                #positive sample
                dfline = coglinkdf.sample(n=1).iloc[0]
                cog1= str(dfline.group1).replace("b",'').replace("'",'').strip()
                cog2= str(dfline.group2).replace("b",'').replace("'",'').strip()
                fam1 = dfline.fam1
                fam2 = dfline.fam2
                #get the features from fitch and profile
                labels = calc_interaction_on_taxonomy(cog1,cog2,tree, bloom)
                
            
            profile1 = profiles[fam1]['tree']
            profile2 = profiles[fam2]['tree']
            presence1 = set([ n.name  for n in profile1.get_leaves() if n.nbr_genes > 0  ])
            presence2 = set([ n.name  for n in profile2.get_leaves() if n.nbr_genes > 0  ])
            #restrict graph to clades where both the proteins are found
            skip = False
            try:
                taxset,matrows,n = getmrca(tree,presence1.intersection(presence2))
            except ValueError:
                #no species overlap
                skip = True    
            if skip == False:
                subconnect_up = sparse2pairs(connectmat_up, matrows)
                subconnect_down = sparse2pairs(connectmat_down, matrows)
                subdiag = sparse2pairs(connectmat_diag, matrows)
                suboverview = overview_connect[matrows,:]                
                suboverview = sparse2pairs(suboverview)
                
                nodefeatures = []
                for i,tp in enumerate([profile1,profile2]):    
                    profilefeatures = np.zeros((template_features.shape[0],3) )
                    #find on which nodes the events happened
                    losses = [ taxindex[n.name]  for n in tp.traverse() if n.lost and n.name in taxset ]
                    dupl = [ taxindex[n.name]  for n in tp.traverse() if n.dupl and n.name in taxset  ]
                    presence = [ taxindex[n.name]  for n in tp.traverse() if n.nbr_genes > 0  and n.name in taxset ]
                    profilefeatures[losses, 0] = 1
                    profilefeatures[dupl, 1] = 1
                    profilefeatures[presence, 2] = 1
                    nodefeatures.append(profilefeatures)
                
                nodefeatures=np.hstack(nodefeatures)
                sub_template_features= template_features[matrows,:]
                sub_node_features= nodefeatures[matrows,:]
                
                sub_node_features = np.hstack([sub_template_features , sub_node_features])
                
                labels = labels[matrows]
                
                
                if verbose == True:
                    print('features',nodefeatures)
                    print( 'labels' , labels)
                
                data = HeteroData()            
                data['phylonodes_up'].x = torch.tensor(sub_node_features, dtype=torch.double )
                data['phylonodes_down'].x = torch.tensor(sub_node_features, dtype=torch.double )
                data['godnode'].x =torch.tensor(  np.zeros((1,sub_node_features.shape[1]))  ,  dtype=torch.double )

                data['phylonodes_up', 'phylolink_up', 'phylonodes_up'].edge_index = torch.tensor(subconnect_up ,  dtype=torch.long )
                data['phylonodes_down', 'phylolink_down', 'phylonodes_down'].edge_index = torch.tensor(subconnect_down ,  dtype=torch.long )             
                data['phylonodes_up', 'phylolink_up_down', 'phylonodes_down'].edge_index = torch.tensor( subdiag ,  dtype=torch.long )
                data['phylonodes_down', 'phylolink_down_up', 'phylonodes_up'].edge_index = torch.tensor( subdiag ,  dtype=torch.long )
                
                data['phylonodes_down', 'informs', 'godnode'].edge_index = torch.tensor(suboverview,  dtype=torch.long )
                data['phylonodes_up', 'informs', 'godnode'].edge_index = torch.tensor(suboverview,  dtype=torch.long )
                                
                #add 2 label classes                
                data['phylonodes_up'].y =torch.tensor(labels  ,  dtype=torch.long )
                data['phylonodes_down'].y =torch.tensor(labels  ,  dtype=torch.long )
                data['godnode'].y =torch.tensor( np.ones((1,))*toss  ,  dtype=torch.long )
                
                data = T.AddSelfLoops()(data)
                data = T.NormalizeFeatures()(data)
                q.put(data)
                #yield data
                
            

In [ ]:
#Traindata = create_data_updown(dendrotree, Datasets['string']['Train'] ,cogfilter , stringprofiles ,   profile_mapper, posi_percent= .5  )
#data = next(Traindata)
#Testdata = create_data_updown(dendrotree, Datasets['string']['Test'] ,cogfilter , stringprofiles ,  profile_mapper , posi_percent= .5  )
#testchunk = [next(Testdata) for i in range(100) ]
#test_dataset = DataLoader(testchunk , batch_size = 1)

import multiprocessing as mp
NCORE = 4

if __name__ == '__main__':
    trainq = mp.Queue(maxsize=10 * NCORE)
    testq = mp.Queue(maxsize=10 * NCORE)
    iolock = mp.Lock()
    pool_train = mp.Pool(NCORE, initializer=create_data_updown, initargs=(trainq, iolock  , 
                dendrotree, Datasets['string']['Train'] ,cogfilter , stringprofiles ,  
                 profile_mapper, .5  ) )
    
    pool_test = mp.Pool(NCORE, initializer=create_data_updown, initargs=(testq, iolock  , 
                dendrotree, Datasets['string']['Test'] ,cogfilter , stringprofiles ,  
                 profile_mapper, .5  ) )
    
    
    data = trainq.get()

In [ ]:
print(data)

In [ ]:
'''
class HeteroGNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_layers):
        super().__init__()
        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HeteroConv({
                
                ('phylonodes_up', 'phylolink', 'phylonodes_up'):SAGEConv((-1,-1), hidden_channels ),
                ('phylonodes_down', 'phylolink', 'phylonodes_down'):SAGEConv((-1,-1), hidden_channels ),
                
                ('phylonodes', 'phylolink', 'phylonodes'):SAGEConv((-1,-1), hidden_channels ),
                ('phylonodes', 'informs', 'godnode'):SAGEConv((-1, -1), hidden_channels),
            }, aggr='sum')
            self.convs.append(conv)
        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        for conv in self.convs:
            
            x_dict = conv(x_dict, edge_index_dict)
            x_dict = {key: x.relu() for key, x in x_dict.items()}
            
            x_dict = {key: self.lin(x) for key, x in x_dict.items()}
        #1d output for each node
        return {key: F.log_softmax(x) for key, x in x_dict.items()}

model = HeteroGNN(hidden_channels=50, out_channels=20, num_layers=2)
model = model.double()
model = model.to(device)'''

In [ ]:
from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv, GATConv, Linear
import torch_geometric.transforms as T

class HeteroGNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_layers):
        super().__init__()
        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HeteroConv({
                ('phylonodes_up', 'phylolink_up', 'phylonodes_up'):SAGEConv((-1,-1), hidden_channels ),
                ('phylonodes_down', 'phylolink_down', 'phylonodes_down'):SAGEConv((-1,-1), hidden_channels ),
                
                ('phylonodes_down', 'phylolink_down_up', 'phylonodes_up'):SAGEConv((-1,-1), hidden_channels ),
                ('phylonodes_up', 'phylolink_up_down', 'phylonodes_down'):SAGEConv((-1,-1), hidden_channels ),
                
                ('phylonodes_up', 'informs', 'godnode'):SAGEConv((-1, -1), hidden_channels),
                ('phylonodes_down', 'informs', 'godnode'):SAGEConv((-1, -1), hidden_channels),
            }, aggr='sum')
            self.convs.append(conv)
        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        for conv in self.convs:
            
            x_dict = conv(x_dict, edge_index_dict)
            x_dict = {key: x.relu() for key, x in x_dict.items()}
            
            x_dict = {key: self.lin(x) for key, x in x_dict.items()}
        #1d output for each node
        return {key: F.log_softmax(x, dim=1) for key, x in x_dict.items()}

model = HeteroGNN(hidden_channels=100, out_channels=20, num_layers=1)
model = model.double()
model = model.to(device)

In [ ]:
data = data.to(device)
with torch.no_grad():  # Initialize lazy modules.
    out = model(data.x_dict ,data.edge_index_dict)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=10**-4)
#optimizer=torch.optim.RMSprop(model.parameters(), lr=0.01, alpha=0.99, eps=1e-08, weight_decay=10**-4, momentum=0.01, centered=False)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

In [ ]:
#model.load_state_dict(torch.load('torch_chkpt.pt'))

In [ ]:
loops = 50
for epoch in range(200):
    
    while trainq.qsize()<100:
        sleep(10)
    datachunk = [trainq.get() for i in range(100) ]
    dataset = DataLoader(datachunk , batch_size =10)
    model.train()
    losses1=[]
    losses2 =[]
    losses3 =[]
    
    for i,data in enumerate(dataset):
        if i % 10 == 0:
            print(i)
        data = data.to(device)
        for loop in range(loops):
            optimizer.zero_grad()
            out = model(data.x_dict ,data.edge_index_dict)
            #loss = F.nll_loss(out, data.y)
            loss1 =  F.nll_loss(out['phylonodes_up'], data['phylonodes_up'].y)
            loss2 =  F.nll_loss(out['phylonodes_down'], data['phylonodes_down'].y)
            loss3 =  F.nll_loss(out['godnode'], data['godnode'].y)

            loss = loss1 + loss2 + loss3
            losses3.append(float(loss3))
            losses2.append(float(loss2))
            losses1.append(float(loss1))
            loss.backward()
            optimizer.step()
        scheduler.step(np.mean(losses2))
    print(f'loss1: {np.mean(losses1):.4f}')
    print(f'loss2: {np.mean(losses2):.4f}')
    print(f'loss3: {np.mean(losses3):.4f}')

    model.eval()
    gncorrects=[]
    ncorrects=[]
    composition_gn = []
    composition_n = []
    
    while trainq.qsize()<100:
        sleep(10)
    testdatachunk = [testq.get() for i in range(100) ]
    testdataset = DataLoader(datachunk , batch_size =10)

    for testdata in test_dataset:
        testdata = testdata.to(device)
        pred = model(testdata.x_dict ,testdata.edge_index_dict)
        pred = {idx:x.to('cpu') for idx,x in pred.items()}
        correct = (pred['godnode'].argmax(dim=1) == testdata['godnode'].y.cpu()).sum()
        
        composition_gn.append(testdata['godnode'].y.cpu().sum() / testdata['godnode'].y.cpu().shape[0] )
        gncorrects.append(correct)
        
        correct = (pred['phylonodes_down'].argmax(dim=1) == testdata['phylonodes_down'].y.cpu()).sum()
        ncorrects.append(correct)
        composition_n.append(testdata['phylonodes_down'].y.cpu().sum() / testdata['phylonodes_down'].y.cpu().shape[0] )
    '''torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, 'torch_chkpt.pt')'''
    
    acc = np.mean(gncorrects) / int(pred['godnode'].cpu().shape[0])
    print(f'gn Accuracy: {acc:.4f}', len(gncorrects), pred['godnode'].shape[0] )
    print(f'gn composition: {np.mean(composition_gn) :.4f}', )
    
    acc = np.mean(ncorrects) / int(pred['phylonodes_down'].cpu().shape[0])
    print(f'n Accuracy: {acc:.4f}', len(ncorrects) , pred['phylonodes_down'].shape[0] )
    print(f'n composition: {np.mean(composition_n) :.4f}', )
    
pool.close()
pool.join()


In [ ]:
model.eval()

preds = []
truths = []
for testdata in test_dataset:
    pred = model(testdata.x_dict ,testdata.edge_index_dict)
    pred = {idx:x.to('cpu') for idx,x in pred.items()}
    
    preds.append(np.array(pred['godnode'] ))
    truths.append(np.array(testdata['godnode'].y.cpu() ))

ROC_curve_single(preds, truths)
    